### Menoplan Fraud Detection Protocol
In this code we merge control and experimental condition data so they are in the same columns. 


In [70]:
import pandas as pd # Pandas is an open source library providing data structures and data analysis tools for Python
import re #regex
from functools import reduce 
import numpy as np
from math import isnan
import matplotlib.pyplot as plt
import urllib.request
import json
from datetime import date
from datetime import datetime
today = 'jan'

In [71]:
beta_file_loc =  "C:/Users/huixin/OneDrive - UC San Diego/menoplan/pilot_phase/merged_analysis/"
pilot_file_loc = "C:/Users/huixin/OneDrive - UC San Diego/menoplan/trial_phase/"
with open(pilot_file_loc+'CColNames.txt') as f:
    CColNames = f.read().splitlines()
with open(pilot_file_loc+'EColNames.txt') as f:
    EColNames = f.read().splitlines()

In [72]:
mo='jan'
allDays = [str(d)+mo for d in range(12,24)]
def getData(dates, surveyPart):
    li=[]
    for d in dates:
        df = pd.read_csv(pilot_file_loc +"qualtrics_data/" + d + "/"+ d +surveyPart + ".csv")
        df = df.iloc[2:] 
        li.append(df)
    joined = pd.concat(li, axis=0, ignore_index=True)
    return joined
p1= getData(allDays, "_p1")
p2 = getData(allDays, "_p2")
qualtrics_data = p2
screening= p1
screening=screening.drop_duplicates()
screening=screening[screening["Email"].str.contains("nghuixin@hotmail.com")==False]
screening=screening[screening["Email"].str.contains("hxng@ucsd.edu")==False]
qualtrics_data=qualtrics_data.drop_duplicates()# delete any duplicates if entire rows are the same

In [73]:
#helper functions
def straightLining(condDf,  rowNum, colNames): #check if answers to questions in a row are the same
    elems = list((condDf[colNames].eq(condDf[colNames].iloc[rowNum:rowNum+1, 0], axis=0).values)[0])
    return elems.count(elems[0]) == len(elems) # are all elements in the list same?

def replaceEmptyWithNAN(varname, dfName): #replace empty string with NAN and convert col to numeric
    dfName[varname].replace(" ", np.NaN, regex=True)
    dfName[varname]=pd.to_numeric(dfName[varname])
    
def findState(zc):
    for dct in js:
        if dct.get("zip_code") == zc:
            return dct.get('state')
        
def containQuoteMarks(string):
    if string.startswith("\"") and string.endswith("\""):
        return True 
def getLsLen(string):
    if string == np.nan:
        li = []
    else:
        li = (string.split(","))
    return len(li)

# #data cleaning
ga_data = pd.read_csv(beta_file_loc + "02nov_merged_pageviews_events_os.csv")
qualtrics_data.columns = qualtrics_data.columns.str.replace('Infromation', 'Information')
qualtrics_data.loc[qualtrics_data['C-Timing_First Click'].isna() , 'Cond'] = 'Exp' #create new col to indicate condition
qualtrics_data.loc[~qualtrics_data['C-Timing_First Click'].isna() , 'Cond'] = 'Ctrl'
qualtrics_data = qualtrics_data.rename(columns= #rename cols
                                       {'RecordedDate': 'EndVisitTs',
                                        'Duration (in seconds)': 'durationMins' }) 

In [74]:
#convert to numeric, replace ' ' with NAN
ls = ['C-Gender', 'E-gender', 'Q_RecaptchaScore','Randomization_2','Randomization_1', 'C-ATT-CHECK1', 
      'C-ATT-CHECK2', 'E-ATT-CHECK1', 'E-ATT-CHECK2',
    #  'C-Age', 'E-Age',  use birthdate var instead
      'durationMins'] #convert to numeric
for x in ls:
    replaceEmptyWithNAN(x, qualtrics_data)
for x in ['Q_RelevantIDDuplicateScore','Q_RelevantIDFraudScore', 'Q_RecaptchaScore' ]:
    replaceEmptyWithNAN(x, screening)   
qualtrics_data['durationMins'] = qualtrics_data['durationMins']/60 #convert duration from secs to mins
first_column =qualtrics_data.pop('Cond')# shift column 'Cond' to first position
qualtrics_data.insert(0, 'Cond', first_column)  
qualtrics_data = qualtrics_data.add_suffix('_p2') #add _p2 to each col to indicate this is part 2 of survey

#### Inattention
1. Unusually short period of time exploring the study websites (&lt; 10 minutes) `p1p2df['StartDate_p2']- p1p2['EndDate']`  
    - Check the timing of when they start answering this question. and when they first entered the survey: Please think about your experience with the information on the website. How much do you agree or disagree with each statement?
2. Unusually short survey completion times (&lt; 2 minutes)  `['durationMins']`
3. Selection of all items in multiple response questions 
    - find variable names of multiple-response questions
    - check if all answers were selected  `selectAll`
4. Selection of same response to all items in a list of possible items (straight lining) 
    - find variable names of multiple-response questions 
        - Check if same `straightLining
5. Failure to answer attention check correctly.`['Q_RecaptchaScore'] >= 80`

#### Duplicate or unusual responses to open-ended items
1. Several responses follow the same pattern in terms of phrasing or format  
    - check if open ended answers are duplicated
2. Nonsensical or non-matching response to open ended questions 
    - check manually
    
#### Other inconsistent or unusual responses
1. High level of nonresponse to survey questions or refusal, >10% - Done 2. Mismatched survey answers (for example 2 different menopause stages, presence/absence of menopause symptoms)  - Done
3. Age out of range (&lt;40, &gt; 55) - Done
4. Male sex - Done
5. Indicated wrong study source - TBA
6. Look to see if strange links are sites that direct people to make money 
    - check traffic from GA for Menoplan Website
7. Duplicate open-ended responses - Done

#### Inconsistent responses to verifiable items
1. Suspicious email address (for example at least 10 random numbers or letters in a row or strange punctuation or capitalization) `['?-Email_p2'].str.isupper()`

2. Time of submission (for example, 12-4 am)
3. Business vs personal address - Manual Check, Spokeo
4. Address not real - Manual Check, Spokeo
5. Address includes quotations marks - Done
6. Name/suffix suspicious - Manual Check, Spokeo
7. Business vs. personal phone number - Manual Check, Spokeo
8. Entered zipcode and state mismatch - Done
9. IP address not in the U.S. - Manual Check, Spokeo 
10. IP address appears on more than one entry 

In [75]:
first30Cols = list(qualtrics_data.iloc[:,:30].columns)
last12Cols = list(qualtrics_data.iloc[:,-12:].columns)

In [76]:
conDf = qualtrics_data[first30Cols + CColNames + last12Cols] #subset only columns relevant to control condition (C-)
expDf = qualtrics_data[first30Cols +EColNames+ last12Cols]


In [77]:
conDf =conDf[conDf['Cond_p2'] == 'Ctrl'] #subset only  rows from control condition participants
expDf =expDf[expDf['Cond_p2'] == 'Exp'] #subset only rows from experiment condition participants

####  Screening Survey
In addition to procedure above, we can also use the relevantID feature from qualtrics, we can find out the likeliness a survey is duplicated.
- https://www.qualtrics.com/support/survey-platform/survey-module/survey-checker/fraud-detection/

In [78]:
cDuplicateScore=screening['Q_RelevantIDDuplicateScore'] >= 75 
cBot=screening['Q_RelevantIDFraudScore'] >= 30
cDuplicateBool=screening['Q_RelevantIDDuplicate']==True
recaptcha_p1 = screening['Q_RecaptchaScore']<0.5
nFraud = cDuplicateScore | cBot| cDuplicateBool|recaptcha_p1
print(str(len(screening)) + ' Total Survey 1 Participants')
print(str(len(screening[nFraud])) + ' Total Survey 1 Fraudsters')
print(str(len(screening[~nFraud])) + ' Total Survey 1 Real Participants')

1224 Total Survey 1 Participants
55 Total Survey 1 Fraudsters
1169 Total Survey 1 Real Participants


In [79]:
expCols = EColNames
conCols = CColNames
varE = []
for x in expCols:
    if x.startswith('E'):
        s = x.replace('E', 'I', 1)
        varE.append(s)
newEColNames = first30Cols+ varE + last12Cols   
        
varC = []
for x in conCols:
    if x.startswith('C'):
        s = x.replace('C', 'I', 1)
        varC.append(s) 
newCColNames = first30Cols + varC + last12Cols  

In [80]:
#create 'selectAll' col that checks if participant selects all options on a multi-choice question
expDf['selectAll']=expDf["E-Peri_Meno_Exper_p2"].apply(lambda x: getLsLen(str(x)))
# check if the zipcode entered matches the state entered (experimental condition)
#expDf['zipcodeState'] = expDf['E-Information_9_p2'].str[0:5].astype('int') #.apply(lambda x: findState(x))#
#expDf['E-Information_8_p2'] = expDf['E-Information_8_p2'].apply(lambda x: str(x)).apply(lambda x: x.upper())
#expDf['zipcodeX_p2']=expDf['E-Information_8_p2'].isin(expDf.zipcodeState) 
#calculate percentage of NAs for each condition
expDf['percentNA_p2'] = expDf.iloc[:,27:].isna().sum(axis = 1)/len(expDf.iloc[:,27:].columns)
#check if street address has quotation marks around
expDf['QuoteMarks']= expDf['E-Information_5_p2'].apply(lambda x: containQuoteMarks(str(x)))
#now we can begin to run the fraud detection procedure 
# recaptcha = expDf['Q_RecaptchaScore_p2'] < 0.8 #recaptcha accuracy less than 80%
# cond1=expDf["E-Peri_Meno_Exper_p2"].str.contains(',11') #select other options even though selected 'I refuse to answer'
# cond2=expDf["E-ATT-CHECK1_p2"] != 5 #Did not select 'Strongly Agree'
# cond3=expDf["E-ATT-CHECK2_p2"] != 1 #Did not select 'Strongly Disagree'
# cond4=expDf['E-Web_Tools_p2'].str.contains(',14')#select other options even though selected 'I refuse to answer'

# cond6 = ~expDf['Randomization_2_p2'].isin(list(np.arange(61, 82, 1)))  #age out of range
# cond7 = ~(expDf['E-gender_p2']).isin([1,4,5]) #not female
# cond8=expDf.duplicated(['IPAddress_p2']) #IP address duplicated
# cond9=expDf['durationMins_p2'] <30 #finished survey in less than 2 mins
# #cond10 = expDf['zipcodeX_p2'] != True #entered zipcode's state does not match entered state
# cond11=expDf['RecipientEmail_p2'].str.isupper()
# cond12 = expDf['selectAll'] == 11

In [81]:
##### create 'selectAll' col that checks if participant selects all options on a multi-choice question
conDf['selectAll']=conDf["C-Peri_Meno_Experie_p2"].apply(lambda x: (getLsLen(x)))
# check if zipcode entered matches the state entered (control condition)
conDf['C-Information_9_p2']= (conDf['C-Information_9_p2'].str[:5])
#conDf['zipcodeState'] = conDf['C-Information_9_p2'].astype('int')# .apply(lambda x: findState(x))
#conDf['C-Information_8_p2'] = conDf['C-Information_8_p2'].apply(lambda x: str(x)).apply(lambda x: x.upper())
#conDf['zipcodeX_p2']=conDf['C-Information_8_p2'].isin(conDf.zipcodeState)
#calculate percentage of NAs for each condition
conDf['percentNA_p2'] = conDf.iloc[:,27:].isna().sum(axis = 1)/len(conDf.iloc[:,27:].columns)

conDf['QuoteMarks']= conDf['C-Information_5_p2'].apply(lambda x: containQuoteMarks(str(x)))

In [82]:
# rename variables so that they are the same to the control's wording 
expDf=expDf.rename(columns= #rename cols
                                       {'I-gender_p2': 'I-Gender_p2',
                                        'I37.UTI_p2': 'I37.UI_p2',
                                       'I-Job_Status_p2': 'I-Job Status_p2',
                                       'I-Peri_Meno_Exper_p2':'I-Peri_Meno_Experie_p2',
                                       'I-Meno_Stage_Tran_p2': 'I-Meno_Stage_Trans_p2',
                                       'I-Menopause_Stage_p2':'I-Meno_Stage_p2'}) 
expDf=expDf.iloc[:,:].set_axis([newEColNames+ ['selectAll', 'percentNA_p2', 'QuoteMarks']], axis=1)
conDf=conDf.iloc[:, :].set_axis([newCColNames+ ['selectAll', 'percentNA_p2', 'QuoteMarks']], axis=1)

#### Which variables are in control condition but not in experimental condition?
`set(varC) - set(varE)
{'I-Email_Link_p2', #specific to control
  'I-Gender_p2',
 'I-Job Status_p2',
 'I-Meno_Stage_Trans_p2',
 'I-Meno_Stage_p2',
 'I-OtherWebsite1_p2', #specific to control
 'I-OtherWebsite2_p2', #specific to control
 'I-Peri_Meno_Experie_p2',
 'I-Website1_p2', #specific to control
 'I-Website2_p2', #specific to control
 'I37.UI_p2'}`

#### Which variables are in experimental  condition but not in  control condition?
`{'I-Job_Status_p2',
 'I-Meno_Stage_Tran_p2',
 'I-Menopause_Stage_p2',
 'I-Peri_Meno_Exper_p2',
 'I-Web_Tools_p2', #specific to exp
 'I-Website_Link_p2', #specific to exp
 'I-gender_p2',
 'I3.1-Useability_p2', #specific to exp
 'I37.UTI_p2'}`

In [83]:
# create csv for today's data
allD = pd.concat([expDf, conDf])
allD.to_csv(pilot_file_loc + "cleaned_data/p2/cleaned" + today + "_p2.csv", index=False)

#### How long does it take for participant to move from first o second survey?
We need to merge the first and second survey using `IPAddress` var to calculate the duration taken to complete the survey. Rename var `IPAddress_p2` to `IPAddress` so that they are mergeable. The merged df is called `p1p2df`. 

In [84]:
screening['EndDate']=pd.to_datetime(screening['EndDate'])#when did they finished survey 1
qualtrics_data['StartDate_p2']=pd.to_datetime(qualtrics_data['StartDate_p2'])#when did they start survey 2
qualtrics_data['EndDate_p2']=pd.to_datetime(qualtrics_data['EndDate_p2'])
#rename column from p2 survey to match columns on IP Address
qualtrics_data=qualtrics_data.rename(columns= {'IPAddress_p2':'IPAddress'})
p1p2df = pd.merge(qualtrics_data, screening, on='IPAddress')  
(p1p2df['StartDate_p2']-p1p2df['EndDate'])#.hist()

0     0 days 00:00:36
1     0 days 00:15:16
2     0 days 00:01:14
3     0 days 00:01:46
4     0 days 00:01:11
            ...      
620   0 days 02:45:24
621   0 days 00:02:50
622   0 days 12:53:16
623   0 days 01:07:10
624   0 days 00:01:32
Length: 625, dtype: timedelta64[ns]